## Action items
1. spark read delta table from adls, use expression to group by for latestchanges.
2. call write api from adb to synapse sql dedicated pool to store latestchanges.
3. auth to sql pool using keyvault

In [1]:
val accountName = "wplushiramsynapseadlsv2"
val accountKey = ""
val containerName = "wplushiramsynapsefs"
val deltaPath = "synapse/workspaces/wplushiramsynapse/warehouse/lastchanges"

In [2]:
val df_delta = spark.read.format("delta").load("abfss://"+containerName+"@"+accountName+".dfs.core.windows.net/"+deltaPath)
df_delta.show()

In [3]:
val latestonly = df_delta.selectExpr("key", "struct(time, newValue, deleted) as otherCols").groupBy("key").agg(max("otherCols").as("latest")).selectExpr("key", "latest.*")
latestonly.show()

In [4]:
//Add required imports
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.SaveMode
import com.microsoft.spark.sqlanalytics.utils.Constants
import org.apache.spark.sql.SqlAnalyticsConnector._

//new write options
val writeOptions:Map[String, String] = Map(Constants.SERVER -> "wplushiramsynapse.sql.azuresynapse.net", 
Constants.TEMP_FOLDER -> "abfss://wplushiramsynapsefs@wplushiramsynapseadlsv2.dfs.core.windows.net/synapse/workspaces/wplushiramsynapse/temp")

In [6]:
//write to Synapse Dedicated SQL Pool (note - default SaveMode is set to ErrorIfExists)
latestonly.
    write.
    options(writeOptions).
    mode(SaveMode.Overwrite).
    synapsesql(tableName = "wplussynapsedw.dbo.latestonly", 
                tableType = Constants.INTERNAL)

## ref.
1. https://docs.databricks.com/data/data-sources/azure/synapse-analytics.html#frequently-asked-questions-faq
2. https://github.com/Azure-Samples/azure-sql-db-databricks/blob/main/notebooks/03b-parallel-switch-in-load-into-partitioned-table-single.ipynb